In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

In [3]:
cd ..

C:\Projects\python\recommender


In [5]:
reg_df = pd.read_csv("./inputs/topcoder/regs.csv")
reg_df.head()

,challengeId,registant
0,30044052,phead
1,30044052,lovefreya
2,30044052,TMALBONPH
3,30044052,daga_sumit
4,30044052,abcivashritt


In [7]:
challenge_counts = reg_df['challengeId'].value_counts()
challenge_counts.head()

30045145    3296
30014146    1024
30046039     818
30048444     789
30063672     666
Name: challengeId, dtype: int64

In [8]:
challenge_counts.size

39916

In [9]:
cha_df = pd.read_csv("./inputs/topcoder/challenge.csv")
cha_df.head()

,challengeId,prize,technologies,platforms
0,30044052,1800.0,['HTML'],['HTML']
1,30044053,1250.0,"['Salesforce', 'Apex', 'Visualforce']","['Force.com', 'Salesforce.com']"
2,30044054,400.0,"['Java', 'MySQL', 'REST']",['AWS']
3,30044055,400.0,['iOS'],['iOS']
4,30044056,400.0,['iOS'],['iOS']


In [10]:
cha_df.shape

(52880, 4)

In [11]:
pd.get_dummies(cha_df['technologies'].apply(pd.Series).stack()).sum(level=0)

,"['.NET 2.0', '.NET 3.0', 'Use Case Diagrams (TCUML)', 'Activity Diagrams (TCUML)', 'Word/Rich Text']","['.NET 2.0', '.NET 3.0']","['.NET 2.0', '.NET System.Addins']","['.NET 2.0', 'ADO.NET']","['.NET 2.0', 'ASP.NET', 'ADO.NET']","['.NET 2.0', 'ASP.NET', 'SQL Server', '.NET 3.0']","['.NET 2.0', 'ASP.NET']","['.NET 2.0', 'SQL Server', 'ADO.NET']","['.NET 2.0', 'SQL Server']","['.NET 2.0', 'Windows Communication Foundation', '.NET 3.0']",...,"['iOS', 'Swift', 'iOS 8.0', 'iOS 8.0']","['iOS', 'Swift', 'iOS 8.0']","['iOS', 'Swift']","['iOS', 'iOS 8.0', 'iOS 8.0']",['iOS'],['jQuery Mobile'],"['jQuery', 'Bootstrap']","['jQuery', 'MongoDB']",['jQuery'],[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
